In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
#!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6
!pip install -r requirements.txt

/content/drive/MyDrive/Colab Notebooks/YOLOv6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.

In [ ]:
!python tools/train.py --batch 32 --conf configs/yolov6s.py --epochs 50 --img-size 640 --data data/data.yaml --device 0

2023-10-14 14:33:48.905848: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 14:33:50.223118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using 1 GPU for training... 
training args are: Namespace(data_path='data/data.yaml', conf_file='configs/yolov6s.py', img_size=640, rect=False, batch_size=32, epochs=50, workers=8, device='0', eval_interval=20, eval_final_only=False, heavy_eval_range=50, check_images=False, check_labels=False, output_dir='./runs/train', name='exp', dist_url='env://', gpu_count=0, local_rank=-1, resume=False, write_trainbatch_tb=False, stop_aug_last_n_epoch=15, save_ckpt_on_last_n_epoch=-1, distill=False, distill_feat=False, quant=False, calib=False, teacher_model_pat

In [ ]:
!python tools/eval.py --data data/data.yaml --weights runs/train/exp1/weights/best_ckpt.pt --device 0

Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab Notebooks/YOLOv6/tools/eval.py", line 16, in <module>
    from yolov6.utils.config import Config
  File "/content/drive/MyDrive/Colab Notebooks/YOLOv6/yolov6/utils/config.py", line 12, in <module>
    from addict import Dict
ModuleNotFoundError: No module named 'addict'


In [ ]:
!python tools/eval.py --data data/data.yaml --batch 32 --weights runs/train/exp1/weights/best_ckpt.pt --task speed #[--half]

Namespace(data='data/data.yaml', weights='runs/train/exp1/weights/best_ckpt.pt', batch_size=32, img_size=640, conf_thres=0.03, iou_thres=0.65, task='speed', device='0', half=False, save_dir='runs/val/', name='exp', shrink_size=0, infer_on_rect=True, reproduce_640_eval=False, eval_config_file='./configs/experiment/eval_640_repro.py', do_coco_metric=True, do_pr_metric=False, plot_curve=True, plot_confusion_matrix=False, verbose=False, config_file='', specific_shape=False, height=None, width=None)
The best conf_thresh when test the speed of the model is larger than 0.4, while you set it to: 0.03
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
Switch model to deploy modality.
Model Summary: Params: 18.50M, Gflops: 45.17
img record infomation path is:/content/drive/MyDrive/Colab Notebooks/data/valid/.images_cache.json
Speed: Checking formats of labels with 2 process(es): 
1043 label(s) found, 0 label(s) missing, 7 label(s) empty, 0 invalid label files: 100% 104

In [ ]:
!python deploy/ONNX/export_onnx.py --weights runs/train/exp1/weights/best_ckpt.pt --device 0 --simplify --batch 32

Namespace(weights='runs/train/exp1/weights/best_ckpt.pt', img_size=[640, 640], batch_size=32, half=False, inplace=False, simplify=True, dynamic_batch=False, end2end=False, trt_version=8, ort=False, with_preprocess=False, topk_all=100, iou_thres=0.65, conf_thres=0.5, device='0')
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
Model(
  (backbone): EfficientRep(
    (stem): RepVGGBlock(
      (nonlinearity): ReLU(inplace=True)
      (se): Identity()
      (rbr_reparam): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (ERBlock_2): Sequential(
      (0): RepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (rbr_reparam): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (1): RepBlock(
        (conv1): RepVGGBlock(
          (nonlinearity): ReLU(inplace=True)
          (se): Identity()
          (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [5]:
!python tools/eval.py --data data/data.yaml --batch 32 --weights runs/train/exp1/weights/best_ckpt.pt --task val --reproduce_640_eval

Namespace(data='data/data.yaml', weights='runs/train/exp1/weights/best_ckpt.pt', batch_size=32, img_size=640, conf_thres=0.03, iou_thres=0.65, task='val', device='0', half=False, save_dir='runs/val/', name='exp', shrink_size=2, infer_on_rect=False, reproduce_640_eval=True, eval_config_file='./configs/experiment/eval_640_repro.py', do_coco_metric=True, do_pr_metric=False, plot_curve=True, plot_confusion_matrix=False, verbose=False, config_file='', specific_shape=False, height=None, width=None)
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
Switch model to deploy modality.
Model Summary: Params: 18.50M, Gflops: 45.17
img record infomation path is:/content/drive/MyDrive/Colab Notebooks/data/valid/.images_cache.json
Val: Checking formats of labels with 2 process(es): 
1043 label(s) found, 0 label(s) missing, 7 label(s) empty, 0 invalid label files: 100% 1043/1043 [03:28<00:00,  5.00it/s]
Convert to COCO format
100% 1043/1043 [00:00<00:00, 180808.39it/s]
Conve